In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
__package__ = "mindlogger_backend_dev.update_girder"
import girder_client
import json
import os
import pandas as pd
import psycopg2
import urllib
from datetime import date
import girder_connections
import object_manipulation
import update_schema

In [3]:
which_girder = "dev"
config, context, api_url = girder_connections.configuration(
    which_girder=which_girder
)
which_girder = "girder-{}".format(which_girder)

In [4]:
girder = girder_connections.connect_to_girder(
    api_url=api_url,
    authentication=(
        config[which_girder]["user"],
        config[which_girder]["password"],
        config[which_girder]["APIkey"]
    ) if "APIkey" in config[which_girder] else (
        config[which_girder]["user"],
        config[which_girder]["password"]
    )
)

Connected to the Girder database 🏗🍃 and authenticated.


In [5]:
girder_connection = girder_client.GirderClient(
    apiUrl=api_url
)

In [6]:
girder_connection.authenticate(config[which_girder]["user"],
            config[which_girder]["password"])

In [7]:
try:
    c = girder_connection.getUser("5b1830ddad97092bdb8e99d1")
except:
    c = girder_connection.getUser("5b4ca32338488311497dc9bc")
c

{'_accessLevel': 2,
 '_id': '5b1830ddad97092bdb8e99d1',
 '_modelType': 'user',
 'admin': True,
 'created': '2018-06-06T19:07:08.791000+00:00',
 'email': 'jon.clucas@childmind.org',
 'emailVerified': True,
 'firstName': 'Jon',
 'groupInvites': [],
 'groups': ['5b2d3884ad97092bdb530b2b',
  '5b2d38a0ad97092bdb530b2c',
  '5b2d38b1ad97092bdb530b2d',
  '5b2d38bcad97092bdb530b2e'],
 'lastName': 'Clucas',
 'login': 'jon.clucas',
 'public': True,
 'size': 0,
 'status': 'enabled'}